In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt

import time

In [2]:
def initializeWeights(n_in, n_out):
    """
    # initializeWeights return the random weights for Neural Network given the
    # number of node in the input layer and output layer

    # Input:
    # n_in: number of nodes of the input layer
    # n_out: number of nodes of the output layer
       
    # Output: 
    # W: matrix of random initial weights with size (n_out x (n_in + 1))"""

    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W



def sigmoid(z):
    """# Notice that z can be a scalar, a vector or a matrix
    # return the sigmoid of input z"""
    
    return (1 / (1 + np.exp(-z)))

def preprocess():
    """ Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set

     Some suggestions for preprocessing step:
     - feature selection"""

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    # Split the training sets into two sets of 50000 randomly sampled training examples and 10000 validation examples. 
    # Your code here.

    # Feature selection
    # Your code here.
    merged_train_data = []
    merged_train_label = []
    merged_test_data = []
    merged_test_label = []
    # yl = np.zeros((10,1))
    for key in mat.keys():
        yl = np.zeros((10,1), dtype= np.uintc)
        if("train" in key):
            merged_train_data.append(mat[key])
            yl[int(key[-1])] = 1
            merged_train_label.append(np.full((len(mat[key]), yl.shape[0]), yl.flatten()))
        if("test" in key):
            merged_test_data.append(mat[key])
            yl[int(key[-1])] = 1
            merged_test_label.append(np.full((len(mat[key]), yl.shape[0]),  yl.flatten()))

    #converting list of arrays to NDarray
    merged_train_data = np.vstack(merged_train_data)
    merged_train_label = np.vstack(merged_train_label)
    test_data = np.vstack(merged_test_data)
    test_label = np.vstack(merged_test_label)

    #feature selection
    column_indices = []
    for i in range(784):
        if(i <= 84 or i>= 700):
            column_indices.append(i)
        if(i % 28 == 0):
            for j in range(i, i+4):
                column_indices.append(j)

    arr = np.array(range(784)).reshape(28,28)
    right_index=[]
    for i in arr:
        for j in range(4):
            column_indices.append(i[-4]+j)

    #cropped data shape 22x20
    merged_train_data =  np.delete(merged_train_data, column_indices, axis = 1)
    test_data = np.delete(test_data, column_indices, axis = 1)
   
    #splitting data randomly
    train_data_height = merged_train_data.shape[0]
    train_data_width = merged_train_data.shape[1]
    random_indices = np.random.choice(range(train_data_height), 60000, replace=False)
    train_data = np.empty((50000, train_data_width))
    train_label = np.empty((50000, merged_train_label.shape[1]), dtype=np.uintc)
    validation_data = np.empty((10000, train_data_width))
    validation_label = np.empty((10000, merged_train_label.shape[1]), dtype=np.uintc)
    for index, randi in enumerate(random_indices[:50000]):
        train_data[index] = merged_train_data[randi]
        train_label[index] = merged_train_label[randi]
    for index, randi in enumerate(random_indices[50000:]):
        validation_data[index] = merged_train_data[randi]
        validation_label[index] = merged_train_label[randi]

    print('preprocess done')

    return train_data, train_label, validation_data, validation_label, test_data, test_label

In [3]:
def nnObjFunction(params, *args):
    """% nnObjFunction computes the value of objective function (negative log 
    %   likelihood error function with regularization) given the parameters 
    %   of Neural Networks, the training data, their corresponding training 
    %   labels and lambda - regularization hyper-parameter.

    % Input:
    % params: vector of weights of 2 matrices w1 (weights of connections from
    %     input layer to hidden layer) and w2 (weights of connections from
    %     hidden layer to output layer) where all of the weights are contained
    %     in a single vector.
    % n_input: number of node in input layer (not include the bias node)
    % n_hidden: number of node in hidden layer (not include the bias node)
    % n_class: number of node in output layer (number of classes in
    %     classification problem
    % training_data: matrix of training data. Each row of this matrix
    %     represents the feature vector of a particular image
    % training_label: the vector of truth label of training images. Each entry
    %     in the vector represents the truth label of its corresponding image.
    % lambda: regularization hyper-parameter. This value is used for fixing the
    %     overfitting problem.
       
    % Output: 
    % obj_val: a scalar value representing value of error function
    % obj_grad: a SINGLE vector of gradient value of error function
    % NOTE: how to compute obj_grad
    % Use backpropagation algorithm to compute the gradient of error function
    % for each weights in weight matrices.

    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    % reshape 'params' vector into 2 matrices of weight w1 and w2
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit j in input 
    %     layer to unit i in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit j in hidden 
    %     layer to unit i in output layer."""

    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args

    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1))) #with bias
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1))) #with bias
    obj_val = 0

    # Your code here
    #normalized
    training_data = np.double(training_data)/255.0
    #adding bias to training data
    training_data = np.concatenate((training_data, np.ones((training_data.shape[0], 1))), axis=1)
    
    #loss function error
    aj = np.matmul(w1, np.transpose(training_data))
    zj = sigmoid(aj)
    # print(f'zj shape:{zj.shape}')
    #adding bias to zj, for the 51th node of hiddden layer
    zj = np.concatenate((zj, np.ones((1, zj.shape[1]))), axis=0)
    # print(f'zj shape with bias:{zj.shape}') 
    bj = np.matmul(w2, zj)
    ol = sigmoid(bj)
    # print(f'ol shape:{ol.shape}')
    
    n = training_data.shape[0]
    j_w1w2 = np.sum(training_label*np.log(ol).T + (1 - training_label)*(np.log(1- ol)).T, axis=None)/(-n)

    # obj_val = j_w1w2 + (lambdaval/(2*n))*(np.sum(np.matmul(w1, np.transpose(w1)), axis=None) + np.sum(np.matmul(w2, np.transpose(w2)), axis=None))
    obj_val = j_w1w2 + (lambdaval/(2*n))*(np.sum(np.square(w1), axis=None) + np.sum(np.square(w2), axis=None))
    print(f'obj_val: {obj_val}')
    #grad error
    zj = zj[:-1]
    w1 = w1[:, :-1]
    w2 = w2[:, :-1]
    training_data = training_data[:, :-1]

    # dj_dw2j = (np.matmul((ol-training_label.T), np.transpose(zj)) + lambdaval*w2)/n
    dj_dw2j = (np.matmul(zj, (ol.T-training_label)).T + (lambdaval*w2))/n
    # dj_dw1jp = (np.matmul(np.matmul((1-zj),zj.T), np.matmul(np.matmul((ol-training_label.T).T,w2).T, training_data)) + lambdaval*w1)/n
    # dj_dw1jp = (np.matmul(((1-zj)*zj).T, np.matmul(np.matmul((ol.T-training_label), w2).T, training_data)) + (lambdaval*w1))/n
    dj_dw1jp = ( np.matmul((((1-zj)*zj).T * np.matmul((ol.T-training_label), w2)).T, training_data) + (lambdaval*w1))/n
    
    dj_dw2j = np.concatenate((dj_dw2j, np.zeros((dj_dw2j.shape[0], 1))), axis=1) #zero added to last column
    dj_dw1jp = np.concatenate((dj_dw1jp, np.zeros((dj_dw1jp.shape[0], 1))), axis=1) #zero added to last column
    obj_grad = np.concatenate((dj_dw1jp.flatten(), dj_dw2j.flatten()), 0)

    # print(f"obj_grad shape: {obj_grad.shape}")
    # print(f'obj_grad len: {obj_grad.shape}')
    # Make sure you reshape the gradient matrices to a 1D array. for instance if your gradient matrices are grad_w1 and grad_w2
    # you would use code similar to the one below to create a flat array
    # obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()),0)
    # obj_grad = np.array([])
    return (obj_val, obj_grad)

def nnPredict(w1, w2, data):
    """% nnPredict predicts the label of data given the parameter w1, w2 of Neural
    % Network.

    % Input:
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit j in input 
    %     layer to unit i in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit j in hidden 
    %     layer to unit i in output layer.
    % data: matrix of data. Each row of this matrix represents the feature 
    %       vector of a particular image
       
    % Output: 
    % label: a column vector of predicted labels"""
    
    # Your code here
    data = np.concatenate((np.ones((data.shape[0], 1)), data), axis=1)

    #loss function error
    aj = np.matmul(w1, np.transpose(data))
    zj = sigmoid(aj)
   
    #adding bias to zj
    zj = np.concatenate((zj, np.ones((1, zj.shape[1]))), axis=0)
    bj = np.matmul(w2, zj)
    ol = sigmoid(bj)
    labels = (ol//ol.max(0)).T
    
    return labels

In [4]:
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

#  Train Neural Network

# set the number of nodes in input unit (not including bias unit)
n_input = train_data.shape[1]

# set the number of nodes in hidden unit (not including bias unit)
n_hidden = 50

# set the number of nodes in output unit
n_class = 10

# initialize the weights into some random matrices
initial_w1 = initializeWeights(n_input, n_hidden)
initial_w2 = initializeWeights(n_hidden, n_class)

# unroll 2 weight matrices into single column vector
initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

# set the regularization hyper-parameter
lambdaval = 60

args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)

# Train Neural Network using fmin_cg or minimize from scipy,optimize module. Check documentation for a working example

opts = {'maxiter': 50}  # Preferred value.

start_time = time.time()
nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
print(">> %.2f seconds" % (time.time() - start_time))

# In Case you want to use fmin_cg, you may have to split the nnObjectFunction to two functions nnObjFunctionVal
# and nnObjGradient. Check documentation for this function before you proceed.
# nn_params, cost = fmin_cg(nnObjFunctionVal, initialWeights, nnObjGradient,args = args, maxiter = 50)

# Reshape nnParams from 1D vector into w1 and w2 matrices
w1 = nn_params.x[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
w2 = nn_params.x[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

# Test the computed parameters

predicted_label = nnPredict(w1, w2, train_data)

# find the accuracy on Training Dataset

print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

predicted_label = nnPredict(w1, w2, validation_data)

# find the accuracy on Validation Dataset

print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

predicted_label = nnPredict(w1, w2, test_data)

# find the accuracy on Validation Dataset

print('\n Test set Accuracy:' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')

preprocess done
obj_val: 6.737117214595212
obj_val: 3.722738139039775
obj_val: 3.6572572706009208
obj_val: 3.266066853143162
obj_val: 3.1744145533493398
obj_val: 2.878670388355919
obj_val: 2.670760076960806
obj_val: 2.1855015645215548
obj_val: 2.0760416531755803
obj_val: 2.0554097887632223
obj_val: 1.933307066830391
obj_val: 1.7145858665336466
obj_val: 1.552892849411979
obj_val: 1.5260740728366677
obj_val: 1.5102169443592819
obj_val: 1.451008017139421
obj_val: 1.4046835470001282
obj_val: 1.3510210930307338
obj_val: 1.3348290362298756
obj_val: 1.3183208096206374
obj_val: 1.2861688865205587
obj_val: 1.2660109336796086
obj_val: 1.233504237851104
obj_val: 1.2245214261503121
obj_val: 1.2038450035640516
obj_val: 1.1878929363030064
obj_val: 1.1404902640456045
obj_val: 1.068118711021914
obj_val: 1.0236410839897139
obj_val: 1.0137030560480116
obj_val: 1.0115579951372924
obj_val: 1.0059789769662766
obj_val: 1.0013581440185209
obj_val: 0.9925955326899459
obj_val: 0.9816269134831948
obj_val: 0.974

C:\Users\JAYPAR~1\AppData\Local\Temp/ipykernel_11984/1244488584.py:23: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-z)))


In [5]:
# n=30, lamda=1
# Training set Accuracy:97.3128%
# Validation set Accuracy:97.18%
# Test set Accuracy:97.322%

# n=30, lamda=60
# Training set Accuracy:97.0188%
# Validation set Accuracy:96.94800000000001%
# Test set Accuracy:97.194%